#### 여러 분류기 조합한 투표 기반 분류기

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
train_input, test_input, train_target, test_target = train_test_split(X, y, random_state=42)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [28]:
lr = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC()
voting = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('svc', svm)],
                         voting='hard')  # 직접 투표(다수결)
voting.fit(train_input, train_target)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [29]:
from sklearn.metrics import accuracy_score
for clf in (lr, rf, svm, voting) :
    clf.fit(train_input, train_target)
    y_pred = clf.predict(test_input)
    print(clf.__class__.__name__, accuracy_score(test_target, y_pred))
    
# voting이 성능이 조금 더 높다.

LogisticRegression 0.864
RandomForestClassifier 0.888
SVC 0.896
VotingClassifier 0.896


In [30]:
# soft voting : 확률이 높은 투표에 비중을 두기에 성능이 더 높다.
lr = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC(probability=True) # 필수 (predict_proba 출력을 위해)
voting = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('svc', svm)],
                         voting='soft')  # 간접 투표(다수결)
voting.fit(train_input, train_target)

for clf in (lr, rf, svm, voting) :
    clf.fit(train_input, train_target)
    y_pred = clf.predict(test_input)
    print(clf.__class__.__name__, accuracy_score(test_target, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.904
SVC 0.896
VotingClassifier 0.912


#### 배깅과 페이스팅
- 같은 알고리즘 사용하고 훈련셋의 서브셋을 무작위로 구성해 분류기를 각기 다르게 학습시키는 것
- 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용

- 배깅 : 훈련셋에서 중복을 허용해 샘플링하는 방식 (부트스르래핑)
- 페이스팅 : 훈련셋에서 중복을 허용하지 않고 샘플링하는 방식

- 수집함수 ; 분류는 통계적 최빈값, 회귀는 평균 계산
- 편향이 크지만, 수집 함수를 통과하면 편향과 분산이 모두 감소(과대적합에 강함)
- 예측기는 모두 동시에 다른 CPU 코어나 서버에서 병렬로 학습가능하다.
- 배깅이 학습 서브셋에 다양성을 증가시키므로 페이스팅보다 편향이 조금 더 높다.
- 다양성을 추가한다는 것은 예측기들의 상관관계를 줄이므로 앙상블의 분산은 감소

In [31]:
# 결정트리 분류기 500개의 앙상블 훈련
# 훈련셋에서 중복을 허용해 무작위로 선택된 100개 샘플(배깅)
# 페이스팅 -> bootstrap=False
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag = BaggingClassifier(
        DecisionTreeClassifier(), n_estimators=500, 
        max_samples = 100, bootstrap=True, n_jobs=-1)

bag.fit(train_input, train_target)
y_pred = bag.predict(test_input)
print(accuracy_score(y_pred, test_target))

0.904


In [32]:
# oob 평가 : oob 샘플 사용하지 않으므로 별도의 검증 세트 사용하지 않고 oob 샘플 사용
# 각 예측기의 oob 평가를 평균
bag = BaggingClassifier(DecisionTreeClassifier(),
                       n_estimators=500, max_samples=100, bootstrap=True,
                       n_jobs=-1, oob_score=True)
bag.fit(train_input, train_target)
bag.oob_score_

0.928

In [33]:
# oob 샘플에 대한 결정 함수 값
bag.oob_decision_function_  # (양, 음)

array([[0.28756477, 0.71243523],
       [0.37532134, 0.62467866],
       [1.        , 0.        ],
       [0.01041667, 0.98958333],
       [0.01813472, 0.98186528],
       [0.10362694, 0.89637306],
       [0.42257218, 0.57742782],
       [0.07594937, 0.92405063],
       [0.93023256, 0.06976744],
       [0.85459184, 0.14540816],
       [0.53018373, 0.46981627],
       [0.04545455, 0.95454545],
       [0.72584856, 0.27415144],
       [0.83830846, 0.16169154],
       [0.89501312, 0.10498688],
       [0.08762887, 0.91237113],
       [0.02319588, 0.97680412],
       [0.93814433, 0.06185567],
       [0.68503937, 0.31496063],
       [0.95717884, 0.04282116],
       [0.05370844, 0.94629156],
       [0.25135135, 0.74864865],
       [0.88917526, 0.11082474],
       [0.99204244, 0.00795756],
       [0.96551724, 0.03448276],
       [0.01578947, 0.98421053],
       [0.9383378 , 0.0616622 ],
       [1.        , 0.        ],
       [0.01767677, 0.98232323],
       [0.7010582 , 0.2989418 ],
       [0.

#### 랜덤 패치와 랜덤 서브스페이스

특성 샘플링 -> 무작위로 선택한 입력 특성의 일부분(고차원에 유용)  
편향을 늘리는 대신 분산을 낮춤

- 랜덤 패치 방식 : 훈련 특성과 샘플을 모두 샘플링
- 랜덤 서브스페이스 방식  
    훈련샘플 모두 사용 (bootstrap=False, max_samples=1.0)  
    특성 샘플링 (bootstrap_features=True)

#### 랜덤 포레스트 - 배깅 or 페이스팅
- max_samples : 훈련셋 크기 지정

- 트리 노드 분할할 때 전체 특성에서 최선의 특성 찾는 (X)  
            무작위로 선택한 특성 후보 중 최적의 특성 찾는 식으로 무작위성 주입  
            => 다양한 트리, 편향을 손해보는 대신 분산 낮춰 훌륭한 모델
            
- 장점 : 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 상대적 중요도 측정이 쉽다. (가중치 평균이며, 각 노드의 가중치는 연관된 훈련 샘플 수와 같다.)   
; 각 결정 트리의 특성 중요도 모두 계산해 더한 후 트리 수로 나눔

* 특성 중요도 : 결정 트리 기반으로 하는 모델은 모두 제공되지만,  
                DecisionTreeClassifier ; 일부 특성 완전히 배제  
                RandomForestClassifier ; 거의 모든 특성에 대해 평가 기회 가짐

In [34]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rf.fit(train_input, train_target)
y_pred = rf.predict(test_input)

In [36]:
# BaggingClassifier 사용
bag = BaggingClassifier(DecisionTreeClassifier(max_features='sqrt',  # 훈련셋크기 = sqrt(500)
                      max_leaf_nodes=16), n_estimators=500)
bag.fit(train_input, train_target)
y_pred_b = bag.predict(test_input)

In [38]:
import numpy as np
np.sum(y_pred_b == y_pred) / len(y_pred_b) # 매우 유사한 예측

0.984

In [39]:
# 특성 중요도
from sklearn.datasets import load_iris
iris = load_iris()
rf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rf.fit(iris['data'], iris['target'])

for name, score in zip(iris['feature_names'], rf.feature_importances_) :
    print(name, score)

sepal length (cm) 0.09515379865969807
sepal width (cm) 0.022643761706333768
petal length (cm) 0.4402919010556954
petal width (cm) 0.4419105385782727


#### 부스팅
- 약한 학습기를 여러 개 연결해 강한 학습기 만드는 앙상블 방법
- 앞의 모델을 보완해나가며 일련의 예측기 학습시킴
- 높은 정확도(편향 줄이기)

- 단점 : 병렬화 불가능 -> 확장성 높지 않다.

In [40]:
# 에이다부스트 : 이전 모델이 과소적합했던(잘못 분류된) 훈련 샘플의 가중치를 더 높이는 것 => 가중치 업데이트
# 반복마다 샘플의 가중치 수정
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200,
                        algorithm='SAMME.R', learning_rate=0.5) # SAMME.R(default) : 클래스 확률 추정
ada.fit(train_input, train_target)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [59]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)
X_new = np.array([[0.8]])

In [60]:
# 그레이디언트 부스팅 : 이전까지의 오차 보정하도록 예측기를 순차적으로 추가
# 이전 예측기가 만든 잔여 오차에 새로운 예측기 학습시킴
from sklearn.tree import DecisionTreeRegressor
tree1 = DecisionTreeRegressor(max_depth=2)
tree1.fit(X, y) # 첫번째 훈련

DecisionTreeRegressor(max_depth=2)

In [61]:
y2 = y - tree.predict(X) # 잔여오차
# 두번째 훈련
tree2 = DecisionTreeRegressor(max_depth=2)
tree2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [62]:
y3 = y2 - tree2.predict(X) # 잔여오차
# 세번째 훈련
tree3 = DecisionTreeRegressor(max_depth=2)
tree3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [63]:
# 새로운 샘플에 대한 예측 만들기 위해 모든 트리의 예측 더함
y_pred = sum(t.predict(X_new) for t in (tree1, tree2, tree3))
print(y_pred)

[0.75026781]


In [64]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3,  # 3번훈련(3개트리)
                                learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [66]:
# 120개 트리로 훈련시키고 최적의 트리수 찾기
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

train_input, val_input, train_target, val_target = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(train_input, train_target)

errors = [mean_squared_error(val_target, y_pred)
         for y_pred in gbrt.staged_predict(val_input)] # 120개

bst_n_estimators = np.argmin(errors) + 1 # index는 0부터 시작하니까

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(train_input, train_target)

GradientBoostingRegressor(max_depth=2, n_estimators=110)

In [76]:
gbrt.predict(val_input)  # 120번째 훈련의 예측값 25개

array([ 0.53922561,  0.33492137,  0.57967684,  0.0215424 ,  0.43974963,
        0.1447461 ,  0.33492137, -0.02912041,  0.0215424 ,  0.67527571,
       -0.02912041,  0.42274462,  0.45491694,  0.7027045 ,  0.05816159,
        0.33492137,  0.1880178 ,  0.58403618,  0.7027045 ,  0.55472294,
        0.42274462,  0.12442881,  0.33492137, -0.02912041,  0.42274462])

In [89]:
# 조기종료
gbrt = GradientBoostingRegressor(max_depth=2, warm_start = True)

min_val_error = float('inf')
error_going_up = 0

for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(train_input, train_target)
    y_pred = gbrt.predict(val_input)
    val_error = mean_squared_error(val_target, y_pred)
    
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stopping

In [90]:
print(gbrt.n_estimators)

72


In [91]:
print("Minimum validation MSE:", min_val_error)

Minimum validation MSE: 0.0028209980081892543


In [100]:
# xgboost
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(train_input, train_target)
y_pred = xgb.predict(val_input)

In [101]:
val_error = mean_squared_error(val_target, y_pred)
print(val_error)

0.003280864635168184


# 연습문제

In [103]:
# 8) MNIST 랜덤포레스트, 엑스트라 트리 분류기, SVM 분류기 훈련시킨 후 간접 또는 직접 투표 방법을 사용해 앙상블로 연결
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

C:\Users\knuyh\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [104]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier


random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [105]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(random_state=42)
Training the ExtraTreesClassifier(random_state=42)
Training the LinearSVC(max_iter=100, random_state=42, tol=20)


C:\Users\knuyh\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Training the MLPClassifier(random_state=42)


In [106]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.9692, 0.9715, 0.859, 0.9606]

In [107]:
from sklearn.ensemble import VotingClassifier

In [108]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

In [111]:
voting_clf = VotingClassifier(named_estimators)

voting_clf.fit(X_train, y_train)
voting_clf.score(X_val, y_val)

C:\Users\knuyh\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


0.97

In [112]:
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

[0.9692, 0.9715, 0.859, 0.9606]

In [113]:
voting_clf.set_params(svm_clf=None)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm_clf', None),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [114]:
voting_clf.estimators

[('random_forest_clf', RandomForestClassifier(random_state=42)),
 ('extra_trees_clf', ExtraTreesClassifier(random_state=42)),
 ('svm_clf', None),
 ('mlp_clf', MLPClassifier(random_state=42))]

In [115]:
del voting_clf.estimators_[2]

In [116]:
voting_clf.score(X_val, y_val)

0.9737

In [117]:
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

0.97

In [118]:
voting_clf.voting = "hard"
voting_clf.score(X_test, y_test)

0.9711

In [119]:
[estimator.score(X_test, y_test) for estimator in voting_clf.estimators_]

[0.9645, 0.9691, 0.9586]

In [120]:
# 9) 검증 세트에서 예측을 만들고 그 결과로 새로운 훈련셋 만들기
# 각 훈련은 하나의 이미지에 대한 전체 분류기의 예측을 담은 벡터이고, 타깃은 이미지의 클래스
# 새로운 훈련셋에 분류기 하나를 훈련시키고, 블렌더를 훈련시키고 분류기를 모아 스태킹 앙상블 구성
# 테스트셋에 앙상블 평가, 블렌더에 예측 주입
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)
    
X_val_predictions

array([[5., 5., 5., 5.],
       [8., 8., 8., 8.],
       [2., 2., 3., 2.],
       ...,
       [7., 7., 7., 7.],
       [6., 6., 6., 6.],
       [7., 7., 7., 7.]], dtype=float32)

In [121]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [122]:
rnd_forest_blender.oob_score_

0.9708

In [123]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [124]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [125]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9701